In [1]:
from MAUtil import *
from MAInit import *
from MACalc import *
from MAPost import *
import re

In [2]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.adsE_database
collection = db.adsE_collection

In [3]:
### get file names ###
env = 'local'
files = os.listdir('./database/')
filenames = []

for i in range(len(files)):
    if 'no' in files[i]:
        if '__' not in files[i]:
            if 'all' not in files[i]:
                filenames.append(files[i])
                print(i, files[i])

3 Pd_111_u1_RPBE_no001_CO_n1_d3.traj
6 Pd_111_u1_RPBE_no002_CO_n1_d3.traj
9 Pd_111_u1_RPBE_no003_CO_n1_d3.traj
12 Pd_111_u1_RPBE_no004_CO_n1_d3.traj
17 Pd_111_u2_RPBE_no001_CO_n1_d9.traj
20 Pd_111_u2_RPBE_no002_CO_n2_d3.traj
23 Pd_111_u2_RPBE_no003_CO_n3_d3.traj
26 Pd_111_u2_RPBE_no004_CO_n4_d3.traj
29 Pd_111_u2_RPBE_no005_CO_n2_d4.traj
32 Pd_111_u2_RPBE_no006_CO_n3_d4.traj
35 Pd_111_u2_RPBE_no007_CO_n2_d4.traj
38 Pd_111_u2_RPBE_no008_CO_n1_d9.traj
41 Pd_111_u2_RPBE_no009_CO_n2_d3.traj
44 Pd_111_u2_RPBE_no010_CO_n3_d3.traj
47 Pd_111_u2_RPBE_no011_CO_n4_d3.traj
50 Pd_111_u2_RPBE_no012_CO_n2_d3.traj
53 Pd_111_u2_RPBE_no013_CO_n2_d3.traj
56 Pd_111_u2_RPBE_no014_CO_n2_d3.traj
59 Pd_111_u2_RPBE_no015_CO_n1_d9.traj
62 Pd_111_u2_RPBE_no016_CO_n2_d3.traj
65 Pd_111_u2_RPBE_no017_CO_n3_d3.traj
68 Pd_111_u2_RPBE_no018_CO_n4_d3.traj
71 Pd_111_u2_RPBE_no019_CO_n2_d4.traj
74 Pd_111_u2_RPBE_no020_CO_n1_d9.traj
77 Pd_111_u2_RPBE_no021_CO_n2_d3.traj
80 Pd_111_u2_RPBE_no022_CO_n3_d3.traj
83 Pd_111_u2_RP

In [10]:
class make_database():
    """
    Create json(python dictionary) object for adsorbate adsorbed surface
    This requires specific filename convention
    ex) Pd_111_u2_RPBE_no015_CO_n1_d9.traj
        (element)_(face)_(u + unitlength)_(xc used in vasp)_(number)_(adsorbate)_
        (n + number of adsorbate)_(d + minimum distance of each adsorbates).traj
    """
    def __init__(self, filename):
        self.filename = filename
        res = re.match('(.*)_(.*)_u(.*)_(.*)_(.*)_(.*)_n(.*)_(.*)(.traj)', filename)
        if not res:
            raise NameError('This file is not for this class.')
        self.numads = int(res.group(7))
        self.iatoms = init_query(filename)
        self.ratoms = query(filename)
        client = MongoClient('localhost', 27017)
        db = client.adsE_database
        self.collection = db.adsE_collection
            
    def make_json(self):
        dic = {}

        res = re.match('(.*)_(.*)_u(.*)_(.*)_(.*)_(.*)_n(.*)_(.*)(.traj)', self.filename)
        ele = res.group(1)
        face = res.group(2)
        unit = int(res.group(3))
        xc = res.group(4)
        ads = res.group(6)
        numads = int(res.group(7))

        res = re.match('(.*)_no.*', self.filename)
        barefile = res.group(1) + '.traj'
        bareatoms = query(barefile, 'spacom')

        adsfile = ads + '_' + xc +'.traj'
        COatoms = query(adsfile, 'scpacom')

        ### calc area ###
        x = self.ratoms.cell[0][0]
        y = self.ratoms.cell[1][1]
        area = x*y

        ### calc surface atom number ###
        tot = 0
        for atom in self.ratoms:
            if atom.tag == 1:
                tot += 1

        ### get energies ###
        ene = self.ratoms.get_potential_energy()
        bareene = bareatoms.get_potential_energy()
        COene = COatoms.get_potential_energy()
        totaladsene = ene - (COene*numads + bareene)

        ### get adsorbate position info ###
        igroups, iposlis, rgroups, rposlis = get_adsorbates_position_info(self.filename, 0)

        ### get COint info ###
        intfile = self.filename[:-5] + '__.traj'
        initatoms = query(intfile, 'spacom')
        intene = initatoms.get_potential_energy() - numads*COene

        ### get COlength info ###
        COlengthlis = []
        for i in range(len(self.ratoms)):
            if self.ratoms[i].symbol == 'C':
                Cpos = self.ratoms[i].position
                Opos = self.ratoms[i+1].position
                COlengthlis.append(np.linalg.norm(Cpos-Opos))

        ### judge valid or not ###

        converged = np.max(np.abs(self.ratoms.get_forces())) < 0.03
        not_moved = get_maximum_movement(self.filename) < 1.5
        kept_sites = igroups == rgroups
        if converged and not_moved and kept_sites:
            isvalid = True
        else:
            isvalid = False

        dic['name'] = self.filename
        dic['isvalid'] = 'yes' if isvalid else 'no'
        dic['element'] = ele
        dic['face'] = face
        dic['unitlength'] = unit
        dic['xc'] = xc
        dic['adsorbate'] = ads
        dic['numberofads'] = numads
        dic['coverage'] = numads/tot
        dic['surfatomnum'] = tot
        dic['E'] = ene
        dic['bareE'] = bareene
        dic['E_CO'] = COene
        dic['totaladsE'] = totaladsene
        dic['aveadsE/suratom'] = totaladsene/tot
        dic['aveadsE/ads'] = totaladsene/numads
        dic['E_int_space'] = intene
        dic['E_each_ads'] = None
        dic['area'] = area
        dic['density'] = numads/area
        dic['igroups'] = igroups
#         dic['iposlis'] = [list(item) for item in iposlis]
        dic['rgroups'] = rgroups
#         dic['rposlis'] = [list(item) for item in rposlis]
        dic['COlengthlis'] = COlengthlis
        dic['converged'] = 'yes' if converged else 'no'
        dic['not_moved'] = 'yes' if not_moved else 'no'
        dic['kept_sites'] = 'yes' if kept_sites else 'no'
        dic['minimum_distance'] = None
        dic['ads_dist2'] = None
        dic['ads_dist3'] = None        

#         for key in dic.keys():
#             print('{}: {}'.format(key, dic[key]))
        print(self.filename)

        return dic
    
    def add_to_database(self):
        if self.check_database():
            print('Already in database')
            return None
        post_data = self.make_json()
        result = collection.insert_one(post_data)
        print('One post: {0}\n'.format(result.inserted_id))
    
    def get_energy_for_each_adsorbates(self):
        data = list(collection.find({'name':self.filename}))
        if len(data) != 1:
            print('More than two data found!')
            return None
        else:
            data = data[0]

        E_each_ads = [] # adsE + slabE

        for site in data['rgroups']:
            refdata = collection.find_one({'element':data['element'], 'face':data['face'], 'unitlength':2,
                                           'xc':data['xc'], 'adsorbate':data['adsorbate'],
                                           'rgroups':[site]})
            if refdata == None:
                print('Reference data is invalid. Each adsorption energy cannnot be calculated.')
                return None
            else:
                E_each_ads.append(refdata['totaladsE'])

        return E_each_ads

    
    def update_database_Eeach(self):
        """
        Assuming more than two adsorbates are on the surface.
        """
        if self.numads <=1:
            raise NameError('This file is not for this fucntion.')
        E_each_ads = self.get_energy_for_each_adsorbates()
        self.collection.find_one_and_update({'name':self.filename},{'$set':{'E_each_ads':E_each_ads}})
        print(self.filename, ' E_each_ads updated.')
        
        
    def update_database_adsorbates_correlation(self, maximumdistance=3):
        """
        Assuming more than two adsorbates are on the surface.
        """
        if self.numads <=1:
            raise NameError('This file is not for this fucntion.')

        if maximumdistance==3:
            repeat = 3
            rratoms = get_repeated_atoms(self.ratoms, repeat)
        else:
            print('Currently maximum distance is 3.')
            
        b_mat, nads = get_coordination_matrix(rratoms, expression=2)
        correlation = get_adsorbates_correlation(b_mat, nads, repeat)
        if correlation[0][1] != 0:
            mindist = 2
        elif correlation[1][1] !=0:
            mindist =3
        else:
            mindist = None
        
        self.collection.find_one_and_update({'name':self.filename},{'$set':{'minimum_distance':mindist}})
        self.collection.find_one_and_update({'name':self.filename},{'$set':{'ads_dist2':correlation[0][1]}})
        self.collection.find_one_and_update({'name':self.filename},{'$set':{'ads_dist3':correlation[1][1]}})
        print(self.filename, 'Adsorbate correlation updated.')
        
    def check_database(self):
        data = list(self.collection.find({'name':self.filename}))
        if not data:
            print('Not in database.')
            return None
        elif len(data) > 1:
            print('More than two data found!')
            return data[0]
        else:
            return data[0]
            
    def delete_from_database(self):
        self.collection.delete_many({'name':self.filename})
        print(self.filename, 'deleted')

In [5]:
for file in filenames:
    try:
        A = make_database(file)
        A.delete_from_database()
        A.add_to_database()
        A.update_database_Eeach()
        A.check_database()
    except:
        print(file, 'Some error occured.')

Pd_111_u1_RPBE_no001_CO_n1_d3.traj deleted
Not in database.
Pd_111_u1_RPBE_no001_CO_n1_d3.traj
One post: 5e598c19b9cec4d627e2fb48

Pd_111_u1_RPBE_no001_CO_n1_d3.traj Some error occured.
Pd_111_u1_RPBE_no002_CO_n1_d3.traj deleted
Not in database.
Pd_111_u1_RPBE_no002_CO_n1_d3.traj
One post: 5e598c19b9cec4d627e2fb4a

Pd_111_u1_RPBE_no002_CO_n1_d3.traj Some error occured.
Pd_111_u1_RPBE_no003_CO_n1_d3.traj deleted
Not in database.
Pd_111_u1_RPBE_no003_CO_n1_d3.traj
One post: 5e598c19b9cec4d627e2fb4c

Pd_111_u1_RPBE_no003_CO_n1_d3.traj Some error occured.
Pd_111_u1_RPBE_no004_CO_n1_d3.traj deleted
Not in database.
Pd_111_u1_RPBE_no004_CO_n1_d3.traj
One post: 5e598c19b9cec4d627e2fb4e

Pd_111_u1_RPBE_no004_CO_n1_d3.traj Some error occured.
Pd_111_u2_RPBE_no001_CO_n1_d9.traj deleted
Not in database.
Pd_111_u2_RPBE_no001_CO_n1_d9.traj
One post: 5e598c19b9cec4d627e2fb50

Pd_111_u2_RPBE_no001_CO_n1_d9.traj Some error occured.
Pd_111_u2_RPBE_no002_CO_n2_d3.traj deleted
Not in database.
Pd_111_u2_

In [6]:
for file in filenames:
    try:
        A = make_database(file)
        print(A.check_database())
    except:
        print(file, 'Some error occured.')

[{'_id': ObjectId('5e598c19b9cec4d627e2fb48'), 'name': 'Pd_111_u1_RPBE_no001_CO_n1_d3.traj', 'isvalid': 'yes', 'element': 'Pd', 'face': '111', 'unitlength': 1, 'xc': 'RPBE', 'adsorbate': 'CO', 'numberofads': 1, 'coverage': 1.0, 'surfatomnum': 1, 'E': -32.30935387, 'bareE': -17.52886733, 'E_CO': -14.48543631, 'totaladsE': -0.2950502300000011, 'aveadsE/suratom': -0.2950502300000011, 'aveadsE/ads': -0.2950502300000011, 'E_int_space': 0.6152077600000005, 'E_each_ads': None, 'area': 6.85643445018277, 'density': 0.14584840083658107, 'igroups': [0], 'rgroups': [0], 'COlengthlis': [1.1612266358898755], 'converged': 'yes', 'not_moved': 'yes', 'kept_sites': 'yes', 'minimum_distance': None, 'ads_dist2': None, 'ads_dist3': None}]
[{'_id': ObjectId('5e598c19b9cec4d627e2fb4a'), 'name': 'Pd_111_u1_RPBE_no002_CO_n1_d3.traj', 'isvalid': 'yes', 'element': 'Pd', 'face': '111', 'unitlength': 1, 'xc': 'RPBE', 'adsorbate': 'CO', 'numberofads': 1, 'coverage': 1.0, 'surfatomnum': 1, 'E': -32.49762181, 'bareE'

[{'_id': ObjectId('5e598c23b9cec4d627e2fb76'), 'name': 'Pd_111_u2_RPBE_no020_CO_n1_d9.traj', 'isvalid': 'no', 'element': 'Pd', 'face': '111', 'unitlength': 2, 'xc': 'RPBE', 'adsorbate': 'CO', 'numberofads': 1, 'coverage': 0.25, 'surfatomnum': 4, 'E': -82.23256805, 'bareE': -70.33920809, 'E_CO': -14.48543631, 'totaladsE': 2.5920763499999993, 'aveadsE/suratom': 0.6480190874999998, 'aveadsE/ads': 2.5920763499999993, 'E_int_space': 0.003775370000001388, 'E_each_ads': None, 'area': 27.42573780073108, 'density': 0.03646210020914527, 'igroups': [3], 'rgroups': [2], 'COlengthlis': [3.57367856458928], 'converged': 'no', 'not_moved': 'no', 'kept_sites': 'no', 'minimum_distance': None, 'ads_dist2': None, 'ads_dist3': None}]
[{'_id': ObjectId('5e598c24b9cec4d627e2fb78'), 'name': 'Pd_111_u2_RPBE_no021_CO_n2_d3.traj', 'isvalid': 'yes', 'element': 'Pd', 'face': '111', 'unitlength': 2, 'xc': 'RPBE', 'adsorbate': 'CO', 'numberofads': 2, 'coverage': 0.5, 'surfatomnum': 4, 'E': -101.72920402, 'bareE': -7

In [7]:
for file in filenames:
    try:
        A = make_database(file)
        A.update_database_adsorbates_correlation()
    except:
        print(file, 'Some error occured.')

Pd_111_u1_RPBE_no001_CO_n1_d3.traj Some error occured.
Pd_111_u1_RPBE_no002_CO_n1_d3.traj Some error occured.
Pd_111_u1_RPBE_no003_CO_n1_d3.traj Some error occured.
Pd_111_u1_RPBE_no004_CO_n1_d3.traj Some error occured.
Pd_111_u2_RPBE_no001_CO_n1_d9.traj Some error occured.
Pd_111_u2_RPBE_no002_CO_n2_d3.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no003_CO_n3_d3.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no004_CO_n4_d3.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no005_CO_n2_d4.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no006_CO_n3_d4.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no007_CO_n2_d4.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no008_CO_n1_d9.traj Some error occured.
Pd_111_u2_RPBE_no009_CO_n2_d3.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no010_CO_n3_d3.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no011_CO_n4_d3.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no012_CO_n2_d3.traj Adsorbate correlation updated.
Pd_111_u2_

In [11]:
for file in filenames:
    try:
        A = make_database(file)
        print(A.check_database()['minimum_distance'])
    except:
        print(file, 'Some error occured.')

None
None
None
None
None
3
3
3
3
2
3
None
2
2
2
2
2
2
None
2
2
2
2
None
2
2
2
None
None
None
None
None
None
